## Parse the GDPR into a dictionary

In [1]:
from legal_parser_utils import parse_legal_texts

legal_texts = parse_legal_texts()
print(legal_texts["GDPR"]["Article 70"]["Point 1"]["Subpoint (b)"])

The Board shall ensure the consistent application of this Regulation. To that end, the Board shall, on its own initiative or, where relevant, at the request of the Commission, in particular: advise the Commission on any issue related to the protection of personal data in the Union, including on any proposed amendment of this Regulation;


## Link different Articles

In [2]:
from legal_parser_utils import find_internal_links_in_legal_text

internal_references = find_internal_links_in_legal_text(legal_texts)
internal_references[:5]

[{'head': 'Article 2', 'tail': 'Article 98'},
 {'head': 'Article 5', 'tail': 'Article 89(1)'},
 {'head': 'Article 5', 'tail': 'Article 89(1)'},
 {'head': 'Article 6', 'tail': 'Article 23(1)'},
 {'head': 'Article 6', 'tail': 'Article 23(1)'}]

## Set up LLM (Not working great rn)

In [3]:
import os
from huggingface_hub import login 
import json
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_community.llms import HuggingFaceEndpoint

token = os.getenv('HF_TOKEN')
login(token=token)
model_name = "mistralai/mixtral-8x7b-instruct-v0.1"

with open("article_linking_few_shot_examples.json", "r") as file:
    few_shot_examples = json.load(file)


example_prompt = PromptTemplate(input_variables=["input", "output"], template="input: {input}\noutput:{output}")

example_prompt.format(**few_shot_examples[0])

prompt = FewShotPromptTemplate(
    examples=few_shot_examples,
    example_prompt=example_prompt,
    prefix= "Return a list consisting of all the articles mentioned in the input.",
    suffix="input: {input}\noutput",
    input_variables=["input"],
)

llm= HuggingFaceEndpoint(repo_id=model_name, max_new_tokens=512, temperature=0.05, repetition_penalty=1.25, huggingfacehub_api_token=token)
chain = prompt|llm

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/dylan/.cache/huggingface/token
Login successful
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.


/home/dylan/miniforge3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  warn_deprecated(


Token is valid (permission: write).
Your token has been saved to /home/dylan/.cache/huggingface/token
Login successful


In [4]:
chain.invoke("The controller shall facilitate the exercise of data subject rights under Articles 15 to 22. 2In the cases referred to in Article 11(2), the controller shall not refuse to act on the request of the data subject for exercising his or her rights under Articles 15 to 22, unless the controller demonstrates that it is not in a position to identify the data subject.")

HfHubHTTPError:  (Request ID: B98ky4wut0dAuRhpos_1f)

403 Forbidden: None.
Cannot access content at: https://api-inference.huggingface.co/models/mistralai/mixtral-8x7b-instruct-v0.1.
If you are trying to create or update content,make sure you have a token with the `write` role.
Access to model mistralai/Mixtral-8x7B-Instruct-v0.1 is restricted and you are not in the authorized list. Visit https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1 to ask for access.

In [9]:
sample_sentence = """As  regards  the  subject  matter,  the  compromise  text  of  Article  1(1)  includes  a  high-level 
statement that one of the purposes of the AI Act  is  to  ensure a high level of protection of 
health, safety  and fundamental  rights  enshrined in the Charter, which includes democracy, 
rule of law and environmental protection. However, all subsequent references in the text to 
the risks  addressed by the Regulation  only include risks to  health, safety and fundamental 
rights, in line with the Council’s mandate. 
Concerning the scope, the compromise text makes it clear that national security is excluded. 
The wording concerning this exclusion in Article 2(3) has been fine tuned to align it more 
closely  with  the  respective  language  used  in  recently  agreed  legal  acts,  such  as  the  Cyber 
Resilience  Act  and  the  Data  Act,  while  the  text  of  the  corresponding  Recital  12a  has 
remained the same as in the Council’s mandate.  """

chain.invoke(sample_sentence)

': []'

## Find mentions of GDPR in the DGA

In [8]:
from legal_parser_utils import find_external_links_in_legal_text

external_references = find_external_links_in_legal_text(legal_texts)
external_references

[{'head': 'DGA: Article 5, Point 6', 'tail': 'GDPR'},
 {'head': 'DGA: Article 9, Point 2', 'tail': 'GDPR'},
 {'head': 'DGA: Article 25, Point 3', 'tail': 'GDPR'}]

## Display the Knowledge Graph

In [9]:
import networkx as nx
from pyvis.network import Network

G = nx.DiGraph()

# Iterate through data to add edges to the graph
for item in internal_references:
    G.add_edge(item["head"], item["tail"])

# Initialize PyVis network
net = Network(notebook=False, height="750px", width="100%")
net.from_nx(G)

for edge in net.edges:
    edge["arrows"] = "to"

# Customize the visualization
net.show_buttons(filter_=['physics'])
net.toggle_physics(True)

# Save the visualization to a html file
net.show("gdpr/gdpr_article_relations.html")

ModuleNotFoundError: No module named 'networkx'

In [ ]:
# Make a KG of external references



## Create a Knowledge Graph for each article

I want it in the form
```python
relations = 
[
    {"Article 1": 
        [
            {"head": "This regulation", "relation": "protects", "tail": "natural persons"},
            ...
        ]
    "Article 2":
    ...
]
    
            

In [ ]:
def find_relations():
    # TODO: Complete
    relations = []

    # pattern = re.compile(r"Article \d+(\(\d+\))?([a-z])?")
    pattern = re.compile(r"Article \d+")

    for article_name, article_dict in legal_texts["GDPR"].items():
        for point_name, point_val in article_dict.items():
            if type(point_val) is dict:
                # It has subpoints
                for subpoint_name, sentence in point_val.items():
                    match = re.search(pattern, sentence)
                    if match:  # This only matches the first one, look into matching multiple Articles
                        references.append({"head": article_name, "tail": match.group()})
                        print(match.group())
                        print(sentence)
            else:
                # It doesn't have subpoints
                match = re.search(pattern, point_val)
                if match:  # This only matches the first one, look into matching multiple Articles
                    references.append({"head": article_name, "tail": match.group()})
                    print(match.group())
                    print(sentence)

    return references
